This notebook provides an environment to apply 
a saved model to new or existing images, and get
outputs of bounding boxes and masks.



In [1]:
import sys
import os
#os.environ['TF_CUDNN_DETERMINISTIC'] = 'true'
os.environ['TENSORPACK_FP16'] = 'true'
sys.path.append('/mask-rcnn-tensorflow/MaskRCNN')
import tensorflow as tf
import tqdm
import numpy as np
import tensorpack.utils.viz as tpviz
from tensorpack import *
from tensorpack.tfutils.common import get_tf_version_tuple
from model.generalized_rcnn import ResNetFPNModel
from config import finalize_configs, config as cfg
from eval import DetectionResult, predict_image, multithread_predict_dataflow, EvalCallback
from performance import ThroughputTracker, humanize_float
from data import get_eval_dataflow, get_train_dataflow, get_batch_train_dataflow
from train import do_visualize, do_evaluate, do_predict
from dataset import DetectionDataset


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.








/usr/local/lib/python3.6/dist-packages/numba/errors.py:131: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
MODEL = ResNetFPNModel(True)
DetectionDataset()

In [3]:
cfg.DATA.BASEDIR = '/data/coco/small_sample/'
#cfg.DATA.BASEDIR = '/home/ec2-user/small_data'
cfg.BACKBONE.WEIGHTS = '/data/coco/pretrained-models/ImageNet-R50-AlignPadding.npz'
finalize_configs(is_training=False)

[1106 17:26:06 @config.py:285] Config: ------------------------------------------
{'BACKBONE': {'FREEZE_AFFINE': False,
              'FREEZE_AT': 2,
              'NORM': 'FreezeBN',
              'RESNET_NUM_BLOCKS': [3, 4, 6, 3],
              'STRIDE_1X1': False,
              'TF_PAD_MODE': False,
              'WEIGHTS': '/data/coco/pretrained-models/ImageNet-R50-AlignPadding.npz'},
 'DATA': {'BASEDIR': '/data/coco/small_sample/',
          'CLASS_NAMES': ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
                          'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
                          'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
                          'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
                          'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
                          'baseball bat', 'baseball glove', 'skatebo

In [4]:
load = '/logs/model-11244.index'
do_visualize(MODEL, load)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
[1106 17:26:06 @dataset.py:50] Instances loaded from /data/coco/small_sample/annotations/instances_train2017.json.


100%|██████████| 25/25 [00:00<00:00, 6040.88it/s]

[1106 17:26:06 @timer.py:50] Load Load annotations for train2017 finished, time:0.0062sec.
[1106 17:26:06 @data.py:130] Ground-Truth Boxes:
| class          |   #box |
|:---------------|-------:|
| BG             |      0 |
| person         |     26 |
| bicycle        |      1 |
| car            |     11 |
| motorcycle     |      1 |
| airplane       |      0 |
| bus            |      2 |
| train          |      0 |
| truck          |      1 |
| boat           |      0 |
| traffic light  |      0 |
| fire hydrant   |      0 |
| stop sign      |      0 |
| parking meter  |      0 |
| bench          |      2 |
| bird           |      6 |
| cat            |      1 |
| dog            |      0 |
| horse          |      0 |
| sheep          |      7 |
| cow            |     13 |
| elephant       |      1 |
| bear           |      0 |
| zebra          |      4 |
| giraffe        |      0 |
| backpack       |      0 |
| umbrella       |      0 |
| handbag        |      2 |
| tie            |  

[1106 17:26:06 @argtools.py:148] WRN "import prctl" failed! Install python-prctl so that processes can be cleaned with guarantee.
[1106 17:26:06 @varmanip.py:196] Checkpoint path /logs/model-11244.index is auto-corrected to /logs/model-11244.





[1106 17:26:06 @registry.py:127] conv0 input: [None, 3, None, None]
Use channels_first data format


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[1106 17:26:07 @registry.py:135] conv0 output: [None, 64, None, None]
[1106 17:26:07 @registry.py:127] pool0 input: [None, 64, None, None]

[1106 17:26:08 @registry.py:135] pool0 output: [None, 64, None, None]
[1106 17:26:08 @registry.py:127] group0/block0/conv1 input: [None, 64, None, None]
Use channels_first data format
[1106 17:26:08 @registry.py:135] group0/block0/conv1 output: [None, 64, None, None]
[1106 17:26:08 @registry.py:127] group0/block0/conv2 input: [None, 64, None, None]
Use channels_first data format
[1106 17:26

[1106 17:26:12 @registry.py:135] group2/block0/convshortcut output: [None, 1024, None, None]
[1106 17:26:12 @registry.py:127] group2/block1/conv1 input: [None, 1024, None, None]
Use channels_first data format
[1106 17:26:12 @registry.py:135] group2/block1/conv1 output: [None, 256, None, None]
[1106 17:26:12 @registry.py:127] group2/block1/conv2 input: [None, 256, None, None]
Use channels_first data format
[1106 17:26:12 @registry.py:135] group2/block1/conv2 output: [None, 256, None, None]
[1106 17:26:12 @registry.py:127] group2/block1/conv3 input: [None, 256, None, None]
Use channels_first data format
[1106 17:26:12 @registry.py:135] group2/block1/conv3 output: [None, 1024, None, None]
[1106 17:26:12 @registry.py:127] group2/block2/conv1 input: [None, 1024, None, None]
Use channels_first data format
[1106 17:26:12 @registry.py:135] group2/block2/conv1 output: [None, 256, None, None]
[1106 17:26:12 @registry.py:127] group2/block2/conv2 input: [None, 256, None, None]
Use channels_first d

[1106 17:26:14 @registry.py:135] fpn/posthoc_3x3_p5 output: [None, 256, None, None]
[1106 17:26:14 @registry.py:127] fpn/maxpool_p6 input: [None, 256, None, None]
[1106 17:26:14 @registry.py:135] fpn/maxpool_p6 output: [None, 256, None, None]
[1106 17:26:14 @registry.py:135] fpn output: [None, 256, None, None],[None, 256, None, None],[None, 256, None, None],[None, 256, None, None],[None, 256, None, None]
[1106 17:26:14 @registry.py:127] rpn input: [None, 256, None, None]
[1106 17:26:14 @registry.py:127] rpn/conv0 input: [None, 256, None, None]
Use channels_first data format
[1106 17:26:14 @registry.py:135] rpn/conv0 output: [None, 256, None, None]
[1106 17:26:14 @registry.py:127] rpn/class input: [None, 256, None, None]
Use channels_first data format
[1106 17:26:14 @registry.py:135] rpn/class output: [None, 3, None, None]
[1106 17:26:14 @registry.py:127] rpn/box input: [None, 256, None, None]
Use channels_first data format
[1106 17:26:14 @registry.py:135] rpn/box output: [None, 12, Non

  3%|▎         | 3/100 [00:04<02:13,  1.37s/it]


AssertionError: Image shape: (1067, 800, 3)
 Boxes:
[[  42  421  145  722]
 [ 389  601  510  749]
 [ 208  411  304  524]
 [  56  468  120  553]
 [ 633  640  713  712]
 [ 196  520  256  581]
 [ 258  905  331 1032]
 [ 653  769  792  889]
 [ 709  541  776  582]
 [ 108  952  196 1057]
 [ 296  517  392  707]
 [  77  469  141  548]
 [ 365  802  496  968]
 [ 426  760  527  894]
 [  67  753  190  923]
 [ 196  918  275 1062]
 [ 744  633  800  713]
 [ 678  645  771  717]
 [ 136  947  204 1034]
 [ 232  822  313  947]
 [  52  956  168 1067]
 [ 164  805  243  897]
 [ 723  589  797  651]
 [ 718  522  770  575]
 [ 195  610  234  683]
 [ 270  494  302  543]
 [ 299  818  377  921]
 [ 186  692  260  746]
 [ 252  518  285  578]
 [ 492  798  708 1026]
 [ 732  593  799  700]
 [  72  883  151  968]
 [ 644  617  765  715]
 [ 232  636  296  699]
 [  65  901  163  996]
 [ 345  685  438  801]
 [ 109  915  185  970]
 [ 678  585  739  643]
 [  84  911  186 1064]
 [ 296    2  343   32]
 [  28  892  207 1051]
 [ 28

In [ ]:
load

In [ ]:
MODEL.get_inference_tensor_names()[0]